# 2. Get data
### 2.1 Read and explore data

In [7]:
import os 
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("data")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"


In [8]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [9]:
def fetch_data(url=HOUSING_URL, path=HOUSING_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    tgz_path = os.path.join(path,"housing.tgz")
    urllib.request.urlretrieve(url, tgz_path)
    data = tarfile.open(tgz_path)
    data.extractall(path=path)
    data.close()

def load_data(path=HOUSING_PATH):
    csv_path = os.path.join(path,'housing.csv')
    return pd.read_csv(csv_path)

In [11]:
os.path.isdir(HOUSING_PATH)

False

In [12]:
try:
    if not os.path.isdir(HOUSING_PATH):
        os.makedirs(HOUSING_PATH)
        fetch_data()
except:
  print('An exception occurred')


housing_data = load_data(path=HOUSING_PATH)
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
